# Simulation of a convolutional encoder and decoder

In this example we will show how to use the convolutional encoder/decoder class in py-itpp. The Viterbi decoder uses the soft received values. Original C++ version of this example is available at http://itpp.sourceforge.net/4.3.1/convcode.html.

In [ ]:
# Imports
import itpp
from matplotlib import pyplot as plt

#  Set up the convolutional encoder/decoder class:
In this example we will simulate a rate 1/3 code that is listed in J. G. Proakis, "Digital communications". The encoder has constraint length 7.

In [ ]:
conv_code = itpp.comm.Convolutional_Code()

generators = itpp.ivec(3)
generators[0] = 91  # Octal 0133
generators[1] = 101 # Octal 0145
generators[2] = 125 # Octal 0175
constraint_length = 7
conv_code.set_generator_polynomials(generators, constraint_length)

# Initialize modulator, channel, and error counter

In [ ]:
# Initialize BPSK modulator
bpsk = itpp.comm.BPSK()

# Initialize an AWGN channel
channel = itpp.comm.AWGN_Channel(noisevar=0)

# Initialize bit error counter
berc = itpp.comm.BERC(indelay=0, inignorefirst=0, inignorelast=0)

# Set up some simulation specific parameters

In [ ]:
EbN0_dB = itpp.vec('-5:0.5:5')
EbN0 = itpp.math.inv_dB(EbN0_dB)

energy_per_bit = 1.0 / conv_code.get_rate()
noise_variance = energy_per_bit * itpp.math.pow(EbN0, -1.0)

nrof_bits = 10000 # Number of bits
max_errors = 100
max_iterations = 10

ber = itpp.vec(EbN0_dB.length()) # Bit error rate
ber.clear()

# Randomize the random number generators
itpp.random.RNG_reset(42)

# Run the simulation

In [ ]:
transmitted_symbols = itpp.vec()
for p in range(EbN0_dB.length()):
    print('Now simulating point %d out of %d with EbN0_dB = %.2f'%(p + 1, EbN0_dB.length(), EbN0_dB[p]))
    berc.clear()  # Clear the bit error rate counter.
    channel.set_noise(noise_variance[p] / 2.0);  # Set the noise value of the AWGN channel.
    for i in range(max_iterations):
      uncoded_bits = itpp.random.randb(nrof_bits)           
      coded_bits = conv_code.encode(uncoded_bits)          
        
      bpsk.modulate_bits(coded_bits, transmitted_symbols)  
      received_symbols = channel(transmitted_symbols)      
        
      decoded_bits = conv_code.decode(received_symbols)    
      berc.count(uncoded_bits, decoded_bits)               
          
      ber[p] = berc.get_errorrate()
      
      # Break the simulation on this point if sufficient number of bit errors were observed
      if (berc.get_errors() > max_errors):
        print('Breaking on point %d with %d errors'%(p + 1, berc.get_errors()))
        break

# Plot results

In [ ]:
# print(ber)
# print(EbN0_dB)

# Plot the results:
plt.figure()
plt.semilogy(EbN0_dB.to_numpy_ndarray(), ber.to_numpy_ndarray())
plt.xlabel('EbN0_dB')
plt.ylabel('BER')
plt.show()